In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df_train = pd.read_csv(r"drive/My Drive/classification_model/df_training.csv")
df_test = pd.read_csv(r"drive/My Drive/classification_model/df_testing.csv")
df_validation = pd.read_csv(r"drive/My Drive/classification_model/df_validation.csv")

In [3]:
#df_train

In [4]:
#df_validation
df_train = df_train.iloc[: , 1:]
df_test = df_test.iloc[: , 1:]
df_validation = df_validation.iloc[: , 1:]

In [5]:
df = pd.concat([df_train, df_test,df_validation])

In [6]:
df = df.iloc[: , 1:]

In [7]:
df

,time,video_b,video_g,video_r,video_h,video_s,video_v,contrast,dissimilarity,homogeneity,energy,correlation,ASM,chroma_stft,zero,chroma_cqt,chroma_cens,melspectrogram,mfcc,rms,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_flatness,spectral_rolloff,poly_features,tonnetz,chroma_stft_sum,chroma_stft_min,chroma_stft_max,chroma_stft_mean,chroma_stft_std,chroma_stft_median,chroma_stft_len,zero_sum,zero_min,zero_max,zero_mean,zero_std,zero_median,...,spectral_bandwidth_median,spectral_bandwidth_len,spectral_contrast_sum,spectral_contrast_min,spectral_contrast_max,spectral_contrast_mean,spectral_contrast_std,spectral_contrast_median,spectral_contrast_len,spectral_flatness_sum,spectral_flatness_min,spectral_flatness_max,spectral_flatness_mean,spectral_flatness_std,spectral_flatness_median,spectral_flatness_len,spectral_rolloff_sum,spectral_rolloff_min,spectral_rolloff_max,spectral_rolloff_mean,spectral_rolloff_std,spectral_rolloff_median,spectral_rolloff_len,poly_features_sum,poly_features_min,poly_features_max,poly_features_mean,poly_features_std,poly_features_median,poly_features_len,tonnetz_sum,tonnetz_min,tonnetz_max,tonnetz_mean,tonnetz_std,tonnetz_median,tonnetz_len,label,mfcc_sum_x,mfcc_sum_y
0,3.053489,91.306924,100.353628,93.507624,58.623848,104.261094,115.588816,74.040973,2.473148,0.720064,0.179346,0.988240,0.095663,[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....,[[0. 0. 0. ... 0. 0. 0.]],[[0.7378403 0.7287309 0.911476 ... 0.39852...,[[0.28798801 0.29058061 0.29390754 ... 0.28008...,[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....,[[-720.0251 -720.0251 -720.0251 ... -720.0251 ...,[[0. 0. 0. ... 0. 0. 0.]],[[0. 0. 0. ... 0. 0. 0.]],[[0. 0. 0. ... 0. 0. 0.]],[[10.11686104 10.11686104 10.11686104 ... 10.1...,[[1.0000007 1.0000007 1.0000007 ... 1.0000007 ...,[[0. 0. 0. ... 0. 0. 0.]],[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....,[[-0.05289372 -0.04136661 0.02710958 ... 0.1...,269351.937500,0.0,1.0,0.374306,0.303606,0.287194,719604,6490.218750,0.0,0.822754,0.108230,0.076259,0.091309,...,2013.640859,59967,8.478020e+06,0.281425,66.432337,20.196870,7.856357,18.299061,419769,1247.392944,0.000007,1.000001,0.020801,0.060761,0.005843,59967,2.310208e+08,0.0,10788.134766,3852.465344,1695.837017,3692.944336,59967,20575.126357,-0.000212,1.822139,0.171554,0.254920,0.000005,119934,-663.988635,-0.584206,0.623592,-0.001845,0.099333,-0.000720,359802,animated,NaN,NaN
1,3.396395,132.009141,127.296736,119.194610,72.646848,92.845192,152.792769,145.736702,3.609109,0.697346,0.175836,0.953497,0.081413,[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....,[[0. 0. 0. ... 0. 0. 0.]],[[0.7267824 0.5933882 0.4627479 ... 0.20757...,[[0.28617507 0.28323629 0.28092294 ... 0.11063...,[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....,[[-531.49457 -531.49457 -531.49457 ... -531.49...,[[0. 0. 0. ... 0. 0. 0.]],[[0. 0. 0. ... 0. 0. 0.]],[[0. 0. 0. ... 0. 0. 0.]],[[10.37575441 10.37575441 10.37575441 ... 10.3...,[[1.0000007 1.0000007 1.0000007 ... 1.0000007 ...,[[0. 0. 0. ... 0. 0. 0.]],[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....,[[ 0.09413428 0.13720254 0.16547466 ... 0.0...,273764.562500,0.0,1.0,0.378531,0.304412,0.296172,723228,7307.005859,0.0,0.709961,0.121240,0.076009,0.105469,...,2206.955877,60269,8.361053e+06,1.647204,63.384726,19.818417,7.436799,17.892850,421883,1783.802612,0.000016,1.000001,0.029597,0.062389,0.012001,60269,2.645233e+08,0.0,10809.667969,4389.044276,1569.649275,4360.473633,60269,174318.547272,-0.001729,15.300068,1.446171,2.031031,0.000123,120538,344.341778,-0.639348,0.635264,0.000952,0.095424,0.002248,361614,animated,NaN,NaN
2,3.332808,121.609755,107.600771,116.679195,86.764124,97.854885,141.985302,93.448568,3.426499,0.642039,0.148755,0.978322,0.060950,[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....,[[0. 0. 0. ... 0. 0. 0.]],[[0.8059579 0.6897122 0.55920815 ... 0.23982...,[[0.22140566 0.22118059 0.22095646 ... 0.32860...,[[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....,[[-516.09283 -516.09283 -516.09283 ... -516.0

In [8]:
df.drop(["chroma_stft","spectral_contrast"	,"zero",	"chroma_cqt",	"chroma_cens",	"melspectrogram",	"mfcc",	"rms"	,"spectral_centroid",	"spectral_bandwidth",		"spectral_flatness",	"spectral_rolloff",	"poly_features",	"tonnetz","mfcc_sum_x","mfcc_sum_y"], axis=1, inplace=True)


In [9]:
df

,time,video_b,video_g,video_r,video_h,video_s,video_v,contrast,dissimilarity,homogeneity,energy,correlation,ASM,chroma_stft_sum,chroma_stft_min,chroma_stft_max,chroma_stft_mean,chroma_stft_std,chroma_stft_median,chroma_stft_len,zero_sum,zero_min,zero_max,zero_mean,zero_std,zero_median,zero_len,n_zeros,chroma_cqt_sum,chroma_cqt_min,chroma_cqt_max,chroma_cqt_mean,chroma_cqt_std,chroma_cqt_median,chroma_cqt_len,chroma_cens_sum,chroma_cens_min,chroma_cens_max,chroma_cens_mean,chroma_cens_std,...,spectral_bandwidth_mean,spectral_bandwidth_std,spectral_bandwidth_median,spectral_bandwidth_len,spectral_contrast_sum,spectral_contrast_min,spectral_contrast_max,spectral_contrast_mean,spectral_contrast_std,spectral_contrast_median,spectral_contrast_len,spectral_flatness_sum,spectral_flatness_min,spectral_flatness_max,spectral_flatness_mean,spectral_flatness_std,spectral_flatness_median,spectral_flatness_len,spectral_rolloff_sum,spectral_rolloff_min,spectral_rolloff_max,spectral_rolloff_mean,spectral_rolloff_std,spectral_rolloff_median,spectral_rolloff_len,poly_features_sum,poly_features_min,poly_features_max,poly_features_mean,poly_features_std,poly_features_median,poly_features_len,tonnetz_sum,tonnetz_min,tonnetz_max,tonnetz_mean,tonnetz_std,tonnetz_median,tonnetz_len,label
0,3.053489,91.306924,100.353628,93.507624,58.623848,104.261094,115.588816,74.040973,2.473148,0.720064,0.179346,0.988240,0.095663,269351.937500,0.0,1.0,0.374306,0.303606,0.287194,719604,6490.218750,0.0,0.822754,0.108230,0.076259,0.091309,59967,145,344613.937500,0.015159,1.0,0.478894,0.269319,0.433784,719604,182238.080255,0.0,0.884380,0.253248,0.138560,...,1993.577241,493.996644,2013.640859,59967,8.478020e+06,0.281425,66.432337,20.196870,7.856357,18.299061,419769,1247.392944,0.000007,1.000001,0.020801,0.060761,0.005843,59967,2.310208e+08,0.0,10788.134766,3852.465344,1695.837017,3692.944336,59967,20575.126357,-0.000212,1.822139,0.171554,0.254920,0.000005,119934,-663.988635,-0.584206,0.623592,-0.001845,0.099333,-0.000720,359802,animated
1,3.396395,132.009141,127.296736,119.194610,72.646848,92.845192,152.792769,145.736702,3.609109,0.697346,0.175836,0.953497,0.081413,273764.562500,0.0,1.0,0.378531,0.304412,0.296172,723228,7307.005859,0.0,0.709961,0.121240,0.076009,0.105469,60269,136,364307.812500,0.011622,1.0,0.503725,0.270494,0.474037,723228,186569.539844,0.0,0.894319,0.257968,0.129561,...,2170.152332,419.106604,2206.955877,60269,8.361053e+06,1.647204,63.384726,19.818417,7.436799,17.892850,421883,1783.802612,0.000016,1.000001,0.029597,0.062389,0.012001,60269,2.645233e+08,0.0,10809.667969,4389.044276,1569.649275,4360.473633,60269,174318.547272,-0.001729,15.300068,1.446171,2.031031,0.000123,120538,344.341778,-0.639348,0.635264,0.000952,0.095424,0.002248,361614,animated
2,3.332808,121.609755,107.600771,116.679195,86.764124,97.854885,141.985302,93.448568,3.426499,0.642039,0.148755,0.978322,0.060950,486532.843750,0.0,1.0,0.370682,0.306044,0.284924,1312536,14015.236816,0.0,0.641113,0.128136,0.075300,0.111816,109378,31,555362.375000,0.012099,1.0,0.423122,0.285788,0.353280,1312536,314422.419303,0.0,0.999303,0.239553,0.161082,...,2540.600468,432.301752,2583.796199,109378,1.499250e+07,1.728419,68.687988,19.581499,6.802504,17.976914,765646,4011.730957,0.000051,1.000001,0.036678,0.048906,0.019256,109378,5.917535e+08,0.0,10831.201172,5410.168954,1595.840263,5523.266602,109378,319131.443052,-0.001340,11.598926,1.458847,1.873690,0.000242,218756,12298.267303,-0.723141,0.710559,0.018740,0.144334,0.011491,656268,animated
3,3.605406,99.874798,106.815739,109.139889,67.424815,106.859563,132.004805,100.044679,3.103249,0.671548,0.176032,0.980199,0.088344,282186.125000,0.0,1.0,0.390318,0.308260,0.309491,722964,7494.262207,0.0,0.800781,0.124392,0.085423,0.103516,60247,146,353887.656250,0.000000,1.0,0.489496,0.273651,0.447190,722964,187829.775653,0.0,0.892805,0.259805,0.125836,...,1964.317232,500.374432,1944.207510,60247,8.582478e+06,1.947302,65.076480,20.350694,7.887025,18.438305,421729,1503.36413

In [11]:
df["label"].replace({"animated": 1, "non-animated": 0}, inplace=True)


In [12]:
df.reset_index(inplace=True,drop=True)

In [13]:
cor = df.corr()


In [14]:
cor.shape

(113, 113)

In [15]:
#cor = df.corr()
cor_target = abs(cor["label"])
relevant_features = cor_target[cor_target>0.4]

In [16]:
relevant_features

video_b                     0.631445
video_g                     0.612190
video_r                     0.580892
video_s                     0.401652
video_v                     0.682697
contrast                    0.430699
chroma_stft_mean            0.579683
chroma_stft_median          0.603620
zero_max                    0.404035
zero_std                    0.440788
chroma_cqt_mean             0.464281
chroma_cqt_median           0.493767
spectral_centroid_std       0.477458
spectral_contrast_mean      0.464656
spectral_contrast_median    0.550253
label                       1.000000
Name: label, dtype: float64

In [17]:
#keep only relevant features
dfff = df[["video_b",                 
"video_g",                    
"video_r",                    
"video_s",                     
"video_v",                     
"contrast" ,                   
"chroma_stft_mean" ,           
"chroma_stft_median" ,         
"zero_max",                    
"zero_std",                    
"chroma_cqt_mean",             
"chroma_cqt_median",           
"spectral_centroid_std",       
"spectral_contrast_mean",      
"spectral_contrast_median" ,   
"label"                       ]]

In [18]:
y_train = dfff["label"]
y_train=y_train.astype('int')
dfff.drop("label", axis=1, inplace=True)

X_train = dfff


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
#from sklearn.model_selection import train_test_split
#x_train,x_test,y_train,y_test = train_test_split(X_train,y_train,test_size = 0.25)


In [20]:
from sklearn.svm import SVC
model = SVC()
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
#scoring = {'acc': 'accuracy'}
cv_accuracy_scores_svc = cross_val_score(model, X_train, y_train, cv=10,scoring='accuracy')
print(cv_accuracy_scores_svc)
print("Mean 10-Fold Accuracy: {}".format(np.mean(cv_accuracy_scores_svc)))

[0.92142857 0.88571429 0.9        0.87857143 0.8057554  0.94964029
 0.76258993 0.86330935 0.84172662 0.84892086]
Mean 10-Fold Accuracy: 0.8657656731757453


In [21]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
clf = DecisionTreeClassifier()

cv_accuracy_scores_rf = cross_val_score(clf, X_train, y_train, cv=10,scoring='accuracy')
print(cv_accuracy_scores_rf)
print("Mean 10-Fold Accuracy: {}".format(np.mean(cv_accuracy_scores_rf)))

[0.89285714 0.87142857 0.86428571 0.87857143 0.87769784 0.94244604
 0.79856115 0.86330935 0.89928058 0.8705036 ]
Mean 10-Fold Accuracy: 0.8758941418293936


In [22]:
from sklearn.ensemble import RandomForestClassifier
clfr=RandomForestClassifier(n_estimators=100)

cv_accuracy_scores_rf = cross_val_score(clfr, X_train, y_train, cv=10,scoring='accuracy')
print(cv_accuracy_scores_rf)
print("Mean 10-Fold Accuracy: {}".format(np.mean(cv_accuracy_scores_rf)))

[0.95       0.92142857 0.85714286 0.89285714 0.90647482 0.98561151
 0.84892086 0.92086331 0.9352518  0.9352518 ]
Mean 10-Fold Accuracy: 0.9153802672147995
